In [1]:
import numpy as np
from scipy.optimize import linprog

In [24]:
import numpy as np

def affine_subspaces_intersection(points1, points2):
    # Calculate the reference points p and q
    p = points1[0]
    q = points2[0]

    # Calculate the direction vectors for both subspaces (A and B matrices)
    A = np.array(points1[1:]) - p
    B = np.array(points2[1:]) - q
    
    # Project q onto the first affine subspace
    A_pseudoinv = np.linalg.pinv(A.T)
    q_proj = p + A.T @ A_pseudoinv @ (q - p)
    
    # Create the matrix C
    C = np.vstack((A, -B))
    C_transpose = C.T

    # Solve the linear system C * z = q - p
    try:
#         z, residuals, rank, s = np.linalg.lstsq(C_transpose, q - p, rcond=None)
        z, residuals, rank, s = np.linalg.lstsq(C_transpose, q_proj - p, rcond=None)
    except ValueError:
        return 'N', None
    
    # Check if the system has a unique solution and if the residuals are close to zero
    if rank == np.linalg.matrix_rank(np.hstack((C.T, (q - p).reshape(-1, 1)))) and np.all(np.isclose(residuals, 0)):
        # Separate t and u from z
        t = z[:A.shape[0]]
        u = z[A.shape[0]:]

        # Calculate the intersection point x
        x = p + A.T @ t
        return 'Y', x
    else:
        return 'N', None

# Example usage
points1 = np.array([
    [0, 0, 0, 0],
    [0, 0, 1, 0], 
    [0, 1, 0, 0]
])

points2 = np.array([
    [0, 3, 8, 0],
    [0, 1, 2, 5], 
    [0, 3, 3, 1],
    [0, -2, 1, 1]
])

intersection_point = affine_subspaces_intersection(points1, points2)
print('intersection_point:', intersection_point)

intersection_point: ('Y', array([ 0.        , -0.12189441,  0.23835404,  0.        ]))


In [19]:
def affine_subspaces_intersection(points1, points2):
    # Calculate the reference points p and q
    p = points1[0]
    q = points2[0]
    print('p:', p)
    print('q:', q)
    print('p-q:', p-q)
    
    # Calculate the direction vectors for both subspaces (A and B matrices)
    A = np.array(points1[1:]) - p
    B = np.array(points2[1:]) - q
    print('A:', A, 'shape', A.shape)
    print('B:', B, 'shape', B.shape)
    
    # Remove linearly dependent vectors from A
    Q_A, R_A = np.linalg.qr(A.T)
    independent_indices_A = np.where(np.abs(np.diagonal(R_A)) > 1e-10)[0]
    A = A[independent_indices_A]
    print('independent_indices_A:', independent_indices_A)
    print('A:', A, 'shape', A.shape)

    # Remove linearly dependent vectors from B
    Q_B, R_B = np.linalg.qr(B.T)
    independent_indices_B = np.where(np.abs(np.diagonal(R_B)) > 1e-10)[0]
    B = B[independent_indices_B]
    print('independent_indices_B:', independent_indices_B)
    print('-B:', -B, 'shape', B.shape)
    
    # Create the matrix C
    C = np.vstack((A, -B))
    C_transpose = C.T
    
    print('C:', C)
    print('C transpose:', C_transpose)
    
    print(C_transpose.shape, ',', (q - p).shape)

    # Solve the linear system C * z = q - p
    z, residuals, rank, s = np.linalg.lstsq(C_transpose, q - p, rcond=None)
    print('z:', z)
    print('residuals:', residuals)
    print('rank:', rank, ', ', C_transpose.shape[1])

    min_rank = min(A.shape[0], B.shape[0])  # <-- Adjusted rank check
    if rank == np.linalg.matrix_rank(np.hstack((C.T, (q - p).reshape(-1, 1)))) and np.all(np.isclose(residuals, 0)):
        # Separate t and u from z
        t = z[:A.shape[0]]
        u = z[A.shape[0]:]
        
        print('t:', t)
        print('u:', u)

        # Calculate the intersection point x
        x = p + A.T @ t
        return x
    else:
        return None

# Example usage
# points1 = np.array([
#     [0, 0, 0],
#     [0, 1, 1],
#     [0, 2, 2]
# ])

# points2 = np.array([
#     [0, 1, 0],
#     [0, 0, 1]
# ])

# points1 = np.array([
#     [0, 0, 0],
#     [0, 1, 1]
# ])

# points2 = np.array([
#     [1, 1, 0],
#     [1, 0, 1]
# ])

points1 = np.array([
    [0, 0, 0, 0],
    [0, 0, 1, 0], 
    [0, 1, 0, 0]
])

points2 = np.array([
    [0, 3, 8, 0],
    [0, 1, 2, 5], 
    [0, 3, 3, 1],
    [0, -2, 1, 1]
])

intersection_point = affine_subspaces_intersection(points1, points2)
print('intersection_point:', intersection_point)


p: [0 0 0 0]
q: [0 3 8 0]
p-q: [ 0 -3 -8  0]
A: [[0 0 1 0]
 [0 1 0 0]] shape (2, 4)
B: [[ 0 -2 -6  5]
 [ 0  0 -5  1]
 [ 0 -5 -7  1]] shape (3, 4)
independent_indices_A: [0 1]
A: [[0 0 1 0]
 [0 1 0 0]] shape (2, 4)
independent_indices_B: [0 1 2]
-B: [[ 0  2  6 -5]
 [ 0  0  5 -1]
 [ 0  5  7 -1]] shape (3, 4)
C: [[ 0  0  1  0]
 [ 0  1  0  0]
 [ 0  2  6 -5]
 [ 0  0  5 -1]
 [ 0  5  7 -1]]
C transpose: [[ 0  0  0  0  0]
 [ 0  1  2  0  5]
 [ 1  0  6  5  7]
 [ 0  0 -5 -1 -1]]
(4, 5) , (4,)
z: [ 0.23835404 -0.12189441 -0.32893375  0.88871636  0.75595238]
residuals: []
rank: 3 ,  5
t: [ 0.23835404 -0.12189441]
u: [-0.32893375  0.88871636  0.75595238]
intersection_point: [ 0.         -0.12189441  0.23835404  0.        ]


In [47]:
1.23259516e-32 < 1e-08

True

In [2]:
def affine_subspace(points):
    # Convert the input list of points to a NumPy array
    points = np.array(points)

    # Check if there are at least two points
    if len(points) < 2:
        raise ValueError("At least two points are required to define an affine subspace.")

    # Calculate the direction vectors by subtracting the first point from the rest of the points
    direction_vectors = points[1:] - points[0]

    # Check if the direction vectors are linearly independent
    rank = np.linalg.matrix_rank(direction_vectors)

    if rank != len(direction_vectors):
        raise ValueError("The direction vectors must be linearly independent.")

    return (points[0], direction_vectors)

In [3]:
def find_intersection(affine_subspace_A, affine_subspace_B):
    origin_A, direction_vectors_A = affine_subspace_A
    origin_B, direction_vectors_B = affine_subspace_B
    
    A = np.block([[direction_vectors_A, -direction_vectors_B]])

    origin_diff = origin_B - origin_A

    # Set the objective function: minimize the L1-norm of the coefficients
    c = np.concatenate((np.ones(direction_vectors_A.shape[1]), np.ones(direction_vectors_B.shape[1])))

    # Set the equality constraint matrix and vector
    A_eq = np.block([[A]])
    b_eq = np.block([origin_diff])

    # Set the bounds for the coefficients
    bounds = [(0, None) for _ in range(direction_vectors_A.shape[1] + direction_vectors_B.shape[1])]

    # Solve the linear program
    res = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds)

    if res.success:
        # If the linear program converges, calculate the intersection point
        intersection_point = origin_A + np.dot(direction_vectors_A, res.x[:direction_vectors_A.shape[1]])
        return intersection_point
    else:
        return None

In [11]:
m = int(input("Enter the dimension of the space (m): "))
n = int(input("Enter the number of points (n): "))

points = []
for _ in range(n):
    point = np.array(list(map(float, input(f"Enter point {_ + 1} ({m} coordinates, separated by spaces): ").split())))
    points.append(point)

origin, direction_vectors = affine_subspace(points)
print("The affine subspace is represented as a translation of the following subspace:")
print(f"Origin: {origin}")
print(f"Direction Vectors: {direction_vectors}")

Enter the dimension of the space (m): 3
Enter the number of points (n): 2
Enter point 1 (3 coordinates, separated by spaces): 0 0 0
Enter point 2 (3 coordinates, separated by spaces): 0 1 1
The affine subspace is represented as a translation of the following subspace:
Origin: [0. 0. 0.]
Direction Vectors: [[0. 1. 1.]]
[[0. 1. 1.]]


In [12]:
m = int(input("Enter the dimension of the space (m): "))
n = int(input("Enter the number of points (n): "))

points = []
for _ in range(n):
    point = np.array(list(map(float, input(f"Enter point {_ + 1} ({m} coordinates, separated by spaces): ").split())))
    points.append(point)

origin_b, direction_vectors_b = affine_subspace(points)
print("The affine subspace is represented as a translation of the following subspace:")
print(f"Origin: {origin}")
print(f"Direction Vectors: {direction_vectors_b}")

Enter the dimension of the space (m): 3
Enter the number of points (n): 2
Enter point 1 (3 coordinates, separated by spaces): 0 1 0
Enter point 2 (3 coordinates, separated by spaces): 0 0 1
The affine subspace is represented as a translation of the following subspace:
Origin: [0. 0. 0.]
Direction Vectors: [[ 0. -1.  1.]]
[[ 0. -1.  1.]]


In [4]:
# Example usage
affine_subspace_A = (np.array([0, 0, 0]), np.array([[0, 1, 1]]))
affine_subspace_B = (np.array([0, 0, 0]), np.array([[0, -1, 1]]))

intersection_point = find_intersection(affine_subspace_A, affine_subspace_B)

if intersection_point is not None:
    print("Intersection Point:", intersection_point)
else:
    print("No intersection found.")

ValueError: Invalid input for linprog: b_eq must be a 1-D array; b_eq must not have more than one non-singleton dimension and the number of rows in A_eq must equal the number of values in b_eq